In [510]:
import pandas as pd
import numpy as np
import re

class PreProcessor:

    def __init__(self, csv_file_path):
        self.__df = pd.read_csv(csv_file_path, index_col= 0)
        self.__del_space()
        self.__df = self.__df.replace(r'^\s*$', np.nan, regex=True) # Заменяем пустые строки в данных на тип NaN
        self.__preprocessing_extra_size_col()
        self.__union_price_cols()
        self.__df = self.__df.replace(r'^\s*$', np.nan, regex=True)

    @property
    def df(self):
        return self.__df
    
    def __del_space(self):
        for col in self.__df.columns: 
            self.__df[col] = self.__df[col].str.strip()
    
    def __preprocessing_extra_size_col(self):
        col_name = 'Доп. размер'
        df = self.__df
        
        df[col_name] = df[col_name].astype(str).str.lower().str.strip()
        df[col_name] = df[col_name].replace(['', 'nan', 'н.д', 'нд', 'н/д', 'с н/д', 'с ост.'], np.nan)

        def extract_all_data(text):
            if pd.isna(text):
                return pd.Series([np.nan, np.nan, np.nan, np.nan], index=['Минимальная_длина', 'Максимальная_длина', 'Упаковка', 'Примечание'])

            original_text = text

            packaging = np.nan
            if 'бухты' in text:
                packaging = 'бухты'
                text = text.replace('бухты', '').strip()
            elif 'размотка' in text:
                packaging = 'размотка'
                text = text.replace('размотка', '').strip()
            elif 'мотки' in text or 'розетты' in text:
                packaging = 'мотки/розетты'
                text = text.replace('мотки', '').replace('розетты', '').strip()
            
            length_primary = np.nan
            length_max = np.nan
            notes = text

            # Вариант 1: Диапазон "число-число" (e.g., "2-6", "3.4-3.7")
            range_match = re.search(r'(\d+\.?\d*)\s*-\s*(\d+\.?\d*)', text)
            if range_match:
                vals = sorted([float(range_match.group(1)), float(range_match.group(2))])
                length_primary, length_max = vals[0], vals[1]
                # Очищаем текст от найденного диапазона для примечаний
                notes = re.sub(r'\d+\.?\d*\s*-\s*\d+\.?\d*', '', text).strip()

            # Вариант 2: Размеры через "х" (e.g., "1.5х10", "1000х1000")
            elif 'х' in text:
                # Эти размеры не являются линейными, поэтому оставляем их в примечаниях
                # а поля длин оставляем пустыми.
                notes = original_text 
                
            # Вариант 3: Префикс "до" (e.g., "до 12")
            elif text.startswith('до '):
                numbers = re.findall(r'\b\d+(?:\.\d+)?\b', text)
                if numbers:
                    length_max = float(numbers[0])
                notes = text
            
            # Вариант 4: Стандартные числа (одиночные или списки)
            else:
                numbers_str = re.findall(r'\b\d+(?:\.\d+)?\b', text)
                if numbers_str:
                    numeric_values = sorted([float(n) for n in numbers_str])
                    if len(numeric_values) == 1:
                        length_primary = length_max = numeric_values[0]
                    elif len(numeric_values) > 1:
                        length_primary = numeric_values[0]
                        length_max = numeric_values[-1]
                    
                    notes = re.sub(r'[\d\.\s,]+', '', text).strip()

            if not notes or notes.isspace():
                notes = np.nan

            return pd.Series([length_primary, length_max, packaging, notes], index=['Минимальная_длина', 'Максимальная_длина', 'Упаковка', 'Примечание'])

        extracted_data = df[col_name].apply(extract_all_data)
        
        for col in extracted_data.columns:
            self.__df[col] = extracted_data[col]

    def __union_price_cols(self):
        
        cols_with_price = [col for col in self.__df.columns if re.search(r"Цена, ", col)]
        price_df = self.__df[cols_with_price]
        
        stacked_prices = price_df.stack()
        
        # Если цен вообще не нашлось, выходим, чтобы не было ошибок
        if stacked_prices.empty:
            self.__df['Цена'] = np.nan
            self.__df['Категория_цены'] = np.nan
            self.__df['Условие_цены'] = np.nan
            self.__df['Звоните'] = False
            self.__df.drop(columns=cols_with_price, inplace=True, errors='ignore')
            return
    
        final_prices = stacked_prices.reset_index()
        final_prices.columns = ['original_index', 'Категория_цены', 'temp_price']
    
        final_prices = final_prices.drop_duplicates(subset='original_index', keep='last')
        
        final_prices = final_prices.set_index('original_index')
        
        self.__df = self.__df.join(final_prices)
    
        PRICECOL = 'Цена'
        CALLCOL = 'Звоните'
        
        is_call = self.__df['temp_price'].str.lower() == 'звоните'
        self.__df[CALLCOL] = is_call.fillna(False)
        
        self.__df.loc[self.__df[CALLCOL], 'temp_price'] = np.nan
    
        pattern = r'^(?P<Цена>[\d\s.,]+)(?:\s+(?P<Условие_цены>.+))?$'
        extracted_data = self.__df['temp_price'].str.extract(pattern)
        
        self.__df[PRICECOL] = extracted_data['Цена'] if not extracted_data.empty else np.nan
        self.__df['Условие_цены'] = extracted_data['Условие_цены'] if not extracted_data.empty else np.nan
        
        self.__df[PRICECOL] = self.__df[PRICECOL].apply(
            lambda x: float(str(x).replace(" ", "").replace(",", ".")) if pd.notna(x) else np.nan
        )
    
        self.__df['Категория_цены'] = self.__df['Категория_цены'].str.replace('Цена, ', '', regex=False)
    
        self.__df.drop(columns=cols_with_price + ['temp_price'], inplace=True, errors='ignore')

In [511]:
import os

a = PreProcessor(os.path.join(os.getcwd(), '23MET_DATA', 'result.csv'))

In [514]:
a.df

,Телефон,Размер,ГОСТ,Наименование,Сталь,Доп. размер,Минимальная_длина,Максимальная_длина,Упаковка,Примечание,Категория_цены,Звоните,Цена,Условие_цены
0,NaN,10,ГОСТ 34028-2016,Арматура А1,А240С,11.7,11.7,11.7,NaN,NaN,р./т,True,NaN,NaN
1,NaN,16,ГОСТ 34028-2016,Арматура А1,А240С,11.7,11.7,11.7,NaN,NaN,р./т,True,NaN,NaN
2,NaN,14,ГОСТ 34028-2016,Арматура А1,А240С,11.7,11.7,11.7,NaN,NaN,р./т,True,NaN,NaN
3,NaN,12,ГОСТ 34028-2016,Арматура А1,А240С,11.7,11.7,11.7,NaN,NaN,р./т,True,NaN,NaN
4,NaN,12,NaN,Арматура А1,А240С,6,6.0,6.0,NaN,NaN,р./т,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53923,NaN,Omnia-46 3,NaN,Электроды,NaN,NaN,NaN,NaN,NaN,NaN,р./кг,True,NaN,NaN
53924,NaN,МР-3С 4,NaN,Электроды 5кг ЛЭЗ,NaN,NaN,NaN,NaN,NaN,NaN,р./кг,True,NaN,NaN
53925,NaN,МР-3С 3,NaN,Электроды 5кг ЛЭЗ,NaN,NaN,NaN,NaN,NaN,NaN,р./кг,True,NaN,NaN
53926,NaN,МР-3С 4,NaN,Электроды ЛЭЗ,NaN,NaN,NaN,NaN,NaN,NaN,р./кг,True,NaN,NaN


In [490]:
# Проверка 
index1 = a.df[a.df['Звоните']].index
index2 = a.df[a.df['Цена'].isna()].index
k = []
for i in index2:
    if i not in index1:
        k.append(i)

a.df.loc[k]

,Телефон,Размер,ГОСТ,Наименование,Сталь,Доп. размер,Минимальная_длина,Максимальная_длина,Упаковка,Примечание,Категория_цены,Звоните,Цена,Условие_цены


In [387]:
import re

pattern = re.compile(
    r'^(?P<Цена>[\d\s.]+)(?:\s+(?P<Условие_цены>.+))?$'
)

test1 = "53 990 1-5т"
test2 = "54 510 >1т"
test3 = '2 185.00'
test4 = '339 900 <0.1т'
test5 = '461.00 100-500м'
test6 = '560'
test7 = '1 072'

for s in [test1, test2, test3, test4, test5, test6, test7]:
    match = pattern.search(s)
    if match:
        print("Цена:", match.group('Цена').replace(' ', ''))
        print("Условие_цены:", match.group('Условие_цены'))

Цена: 53990
Условие_цены: 1-5т
Цена: 54510
Условие_цены: >1т
Цена: 2185.00
Условие_цены: None
Цена: 339900
Условие_цены: <0.1т
Цена: 461.00
Условие_цены: 100-500м
Цена: 560
Условие_цены: None
Цена: 1072
Условие_цены: None


In [377]:
import re
pattern = re.compile(pattern= r'^(?P<Цена>[\d\s]+)\s+(?P<Условие_цены>[><\d-]+.*[тТ]?)')
match = pattern.search(price3)
if match:
    print(int(match.group('Цена').replace(' ', '_')))
    print(match.group('Условие_цены'))

2
185.00


In [ ]:
a.df[:10000].to_csv(os.getcwd() + "/test.csv")